In [23]:
from pinecone import Pinecone,ServerlessSpec
import os
from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv(),override=True)
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

In [42]:
# List existing indexes
try:
    existing_indexes = pc.list_indexes()
    print("Existing indexes:", existing_indexes)
    index_names = [idx.name for idx in existing_indexes.indexes] if hasattr(existing_indexes, 'indexes') else []
except Exception as e:
    print(f"Error listing indexes: {e}")
    index_names = []

Existing indexes: []


In [43]:
# Only create index if it doesn't exist

index_name = "testindex"
if index_name not in index_names:
    print(f"Creating index '{index_name}'...")
    try:
        pc.create_index(
            name=index_name,
            dimension=384,
            metric='cosine',
            spec=ServerlessSpec(cloud='aws', region='us-east-1')
        )
        print(f"Index '{index_name}' created successfully!")
    except Exception as e:
        print(f"Error creating index: {e}")
        print(f"Error type: {type(e).__name__}")
else:
    print(f"Index '{index_name}' already exists!")

Creating index 'testindex'...
Index 'testindex' created successfully!


Insert embedding in pinecone

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings 
from langchain_pinecone import PineconeVectorStore
load_dotenv(find_dotenv(),override=True)

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
index=pc.Index(index_name)
vectors_to_upsert = [
	{"id": "vec4", "values": embeddings.embed_query("my name is ram"), "metadata": {"text": "my nais ram"}},
	{"id": "vec5", "values": embeddings.embed_query("i am living in india"), "metadata": {"text": "i am living in india"}},
	{"id": "vec2", "values": embeddings.embed_query("example text for vec2"), "metadata": {"text": "example text for vec2"}},
	{"id": "vec3", "values": embeddings.embed_query("example text for vec3"), "metadata": {"text": "example text for vec3"}},
]

index.upsert(vectors=vectors_to_upsert, namespace="test-namespace")

In [54]:
from langchain_huggingface import HuggingFaceEmbeddings 
from langchain_pinecone import PineconeVectorStore
load_dotenv(find_dotenv(),override=True)

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = PineconeVectorStore(index_name="testindex", embedding=embeddings, namespace="test-namespace")
docs=vectorstore.similarity_search("india",include_metadata=False,k=1)
# len(docs)
docs[0]

Document(id='vec5', metadata={}, page_content='i am living in india')